# API City prices for 1 bedroom apartment in city centre from numbeo.com

In [36]:
# import libraries
import requests
import os
import pandas as pd
import json
import sql_functions as sf

In [12]:
# To be able to pull the API key from the .env, we need to import load_dotenv

from dotenv import load_dotenv
load_dotenv()

True

Info from API numbeo.com

yearLastUpdate': 2023

'item_id': 26, 'item_name': 'Apartment (1 bedroom) in City Centre, Rent Per Month'


In [54]:
#Load city list from data/cities_numbeo excel file
city_list = pd.read_excel('data/Cities_numbeo.xlsx')


In [55]:
city_list.head()

,City
0,"Aachen, Germany"
1,"Aalborg, Denmark"
2,"Aberdeen, United Kingdom"
3,"Abidjan, Ivory Coast"
4,"Abu Dhabi, United Arab Emirates"


In [56]:
#Split the list in three columns on comma ' , ' 
city_list[['city', 'country', 'country_2']] = city_list['City'].str.split(', ', expand=True)

In [61]:
city_list.loc[city_list['country_2'].notnull(), 'country'] = city_list['country_2']

In [62]:
city_list

,City,city,country,country_2
0,"Aachen, Germany",Aachen,Germany,None
1,"Aalborg, Denmark",Aalborg,Denmark,None
2,"Aberdeen, United Kingdom",Aberdeen,United Kingdom,None
3,"Abidjan, Ivory Coast",Abidjan,Ivory Coast,None
4,"Abu Dhabi, United Arab Emirates",Abu Dhabi,United Arab Emirates,None
...,...,...,...,...
549,"Zadar, Croatia",Zadar,Croatia,None
550,"Zagreb, Croatia",Zagreb,Croatia,None
551,"Zaragoza (Saragossa), Spain",Zaragoza (Saragossa),Spain,None
552,"Zug, Switzerland",Zug,Switzerland,None


In [65]:
# Check the result
city_list['country'].unique()

array(['Germany', 'Denmark', 'United Kingdom', 'Ivory Coast',
       'United Arab Emirates', 'Ghana', 'Saudi Arabia', 'Turkey',
       'Ethiopia', 'Australia', 'Morocco', 'India', 'United States',
       'Egypt', 'Algeria', 'Spain', 'Kazakhstan', 'Jordan', 'Netherlands',
       'Belgium', 'Paraguay', 'Greece', 'New Zealand', 'Portugal', 'Iraq',
       'Philippines', 'Azerbaijan', 'Indonesia', 'Thailand',
       'Bosnia And Herzegovina', 'Barbados', 'Italy', 'Colombia',
       'Canada', 'Switzerland', 'Georgia', 'China', 'Lebanon', 'Serbia',
       'Brazil', 'Norway', 'Kyrgyzstan', 'North Macedonia', 'France',
       'Romania', 'Slovakia', 'Czech Republic', 'Hungary', 'Montenegro',
       'Argentina', 'Bulgaria', 'Mexico', 'South Africa', 'Venezuela',
       'Moldova', 'Bangladesh', 'Sri Lanka', 'Ireland', 'Ecuador',
       'Vietnam', 'Senegal', 'Syria', 'Tanzania', 'Ukraine', 'Qatar',
       'Cameroon', 'Isle Of Man', 'Croatia', 'Finland', 'Botswana',
       'Poland', 'Cayman Islands',

In [67]:
# Drop the column City and country_2
city_list.drop(['City', 'country_2'], axis = 1, inplace = True)

In [85]:
city_list.head()

,city,country
0,Aachen,Germany
1,Aalborg,Denmark
2,Aberdeen,United Kingdom
3,Abidjan,Ivory Coast
4,Abu Dhabi,United Arab Emirates


In [86]:
# Make a list for API query
city = list(city_list['city'])

In [87]:
city

['Aachen',
 'Aalborg',
 'Aberdeen',
 'Abidjan',
 'Abu Dhabi',
 'Accra',
 'Ad Dammam',
 'Adana',
 'Addis Ababa',
 'Adelaide',
 'Agadir',
 'Ahmedabad',
 'Ajman',
 'Al Khobar',
 'Alanya',
 'Albuquerque',
 'Alexandria',
 'Algiers',
 'Alicante',
 'Almaty',
 'Amman',
 'Amsterdam',
 'Ankara',
 'Antalya',
 'Antwerp',
 'Arhus',
 'Astana (Nur-Sultan)',
 'Asuncion',
 'Athens',
 'Atlanta',
 'Auckland',
 'Augsburg',
 'Austin',
 'Aveiro',
 'Baghdad',
 'Baguio',
 'Baku',
 'Bali',
 'Bandung',
 'Bangalore',
 'Bangkok',
 'Banja Luka',
 'Barbados',
 'Barcelona',
 'Bari',
 'Barranquilla',
 'Barrie',
 'Basel',
 'Batumi',
 'Beijing',
 'Beirut',
 'Belfast',
 'Belgrade',
 'Belo Horizonte',
 'Bergamo',
 'Bergen',
 'Berlin',
 'Bern',
 'Bhubaneswar',
 'Bilbao',
 'Birmingham',
 'Bishkek',
 'Bitola',
 'Bogota',
 'Boise',
 'Bologna',
 'Bonn',
 'Bordeaux',
 'Boston',
 'Bournemouth',
 'Braga',
 'Brampton',
 'Brasilia',
 'Brasov',
 'Bratislava',
 'Bremen',
 'Brescia',
 'Brighton',
 'Brisbane',
 'Bristol',
 'Brno',
 'B

In [88]:
# Make a country list for API query
country = list(city_list['country'])

In [89]:
country

['Germany',
 'Denmark',
 'United Kingdom',
 'Ivory Coast',
 'United Arab Emirates',
 'Ghana',
 'Saudi Arabia',
 'Turkey',
 'Ethiopia',
 'Australia',
 'Morocco',
 'India',
 'United Arab Emirates',
 'Saudi Arabia',
 'Turkey',
 'United States',
 'Egypt',
 'Algeria',
 'Spain',
 'Kazakhstan',
 'Jordan',
 'Netherlands',
 'Turkey',
 'Turkey',
 'Belgium',
 'Denmark',
 'Kazakhstan',
 'Paraguay',
 'Greece',
 'United States',
 'New Zealand',
 'Germany',
 'United States',
 'Portugal',
 'Iraq',
 'Philippines',
 'Azerbaijan',
 'Indonesia',
 'Indonesia',
 'India',
 'Thailand',
 'Bosnia And Herzegovina',
 'Barbados',
 'Spain',
 'Italy',
 'Colombia',
 'Canada',
 'Switzerland',
 'Georgia',
 'China',
 'Lebanon',
 'United Kingdom',
 'Serbia',
 'Brazil',
 'Italy',
 'Norway',
 'Germany',
 'Switzerland',
 'India',
 'Spain',
 'United Kingdom',
 'Kyrgyzstan',
 'North Macedonia',
 'Colombia',
 'United States',
 'Italy',
 'Germany',
 'France',
 'United States',
 'United Kingdom',
 'Portugal',
 'Canada',
 'Brazil

In [90]:
#Create a dictionary city, country for API query
city_count_dict = dict(zip(city, country))

In [91]:
city_count_dict

{'Aachen': 'Germany',
 'Aalborg': 'Denmark',
 'Aberdeen': 'United Kingdom',
 'Abidjan': 'Ivory Coast',
 'Abu Dhabi': 'United Arab Emirates',
 'Accra': 'Ghana',
 'Ad Dammam': 'Saudi Arabia',
 'Adana': 'Turkey',
 'Addis Ababa': 'Ethiopia',
 'Adelaide': 'Australia',
 'Agadir': 'Morocco',
 'Ahmedabad': 'India',
 'Ajman': 'United Arab Emirates',
 'Al Khobar': 'Saudi Arabia',
 'Alanya': 'Turkey',
 'Albuquerque': 'United States',
 'Alexandria': 'Egypt',
 'Algiers': 'Algeria',
 'Alicante': 'Spain',
 'Almaty': 'Kazakhstan',
 'Amman': 'Jordan',
 'Amsterdam': 'Netherlands',
 'Ankara': 'Turkey',
 'Antalya': 'Turkey',
 'Antwerp': 'Belgium',
 'Arhus': 'Denmark',
 'Astana (Nur-Sultan)': 'Kazakhstan',
 'Asuncion': 'Paraguay',
 'Athens': 'Greece',
 'Atlanta': 'United States',
 'Auckland': 'New Zealand',
 'Augsburg': 'Germany',
 'Austin': 'United States',
 'Aveiro': 'Portugal',
 'Baghdad': 'Iraq',
 'Baguio': 'Philippines',
 'Baku': 'Azerbaijan',
 'Bali': 'Indonesia',
 'Bandung': 'Indonesia',
 'Bangalore

In [70]:
city

['Aachen',
 'Aalborg',
 'Aberdeen',
 'Abidjan',
 'Abu Dhabi',
 'Accra',
 'Ad Dammam',
 'Adana',
 'Addis Ababa',
 'Adelaide',
 'Agadir',
 'Ahmedabad',
 'Ajman',
 'Al Khobar',
 'Alanya',
 'Albuquerque',
 'Alexandria',
 'Algiers',
 'Alicante',
 'Almaty',
 'Amman',
 'Amsterdam',
 'Ankara',
 'Antalya',
 'Antwerp',
 'Arhus',
 'Astana (Nur-Sultan)',
 'Asuncion',
 'Athens',
 'Atlanta',
 'Auckland',
 'Augsburg',
 'Austin',
 'Aveiro',
 'Baghdad',
 'Baguio',
 'Baku',
 'Bali',
 'Bandung',
 'Bangalore',
 'Bangkok',
 'Banja Luka',
 'Barbados',
 'Barcelona',
 'Bari',
 'Barranquilla',
 'Barrie',
 'Basel',
 'Batumi',
 'Beijing',
 'Beirut',
 'Belfast',
 'Belgrade',
 'Belo Horizonte',
 'Bergamo',
 'Bergen',
 'Berlin',
 'Bern',
 'Bhubaneswar',
 'Bilbao',
 'Birmingham',
 'Bishkek',
 'Bitola',
 'Bogota',
 'Boise',
 'Bologna',
 'Bonn',
 'Bordeaux',
 'Boston',
 'Bournemouth',
 'Braga',
 'Brampton',
 'Brasilia',
 'Brasov',
 'Bratislava',
 'Bremen',
 'Brescia',
 'Brighton',
 'Brisbane',
 'Bristol',
 'Brno',
 'B

In [71]:
# #API download from www.numbeo.com 
url = 'https://www.numbeo.com/api/city_prices'
api_key = 'zlueew978sczoi'
#country_list ' don`t need, we take the values for countries from country_list in cell above 
desired_item_ids = [26]
currency = "EUR"

data_list = []

for city in city:
    # Send the HTTP GET request for each country with currency parameter
    response = requests.get(url, params={"api_key": api_key, "query": city, "currency": currency})
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Access the "prices" list from the response
        prices = data.get("prices", [])
        
        # Access the country's land information
        city = city
        
        # Initialize a list to store the desired items for the country
        desired_items = []
        
        # Iterate over the prices and check if the item_id matches the desired values
        for price in prices:
            item_id = price.get("item_id")
            if item_id in desired_item_ids:
                desired_items.append(price)
        
        # Add the desired items, land information, and currency to the data list
        for item in desired_items:
            item["city"] = city
            item["currency"] = currency
        data_list.extend(desired_items)
    else:
        print(f"Request for {city} failed with status code:", response.status_code)

# Create a DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the DataFrame
df

,data_points,item_id,lowest_price,average_price,highest_price,item_name,city,currency
0,14,26,450.000000,561.444444,850.000000,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,EUR
1,10,26,537.070588,726.963403,859.312941,"Apartment (1 bedroom) in City Centre, Rent Per...",Aalborg,EUR
2,24,26,550.714099,640.494990,811.578672,"Apartment (1 bedroom) in City Centre, Rent Per...",Aberdeen,EUR
3,14,26,228.673485,491.224523,914.693940,"Apartment (1 bedroom) in City Centre, Rent Per...",Abidjan,EUR
4,12,26,79.536758,674.859546,1477.416411,"Apartment (1 bedroom) in City Centre, Rent Per...",Accra,EUR
...,...,...,...,...,...,...,...,...
519,70,26,708.136823,992.284471,1377.994500,"Apartment (1 bedroom) in City Centre, Rent Per...",Yerevan,EUR
520,10,26,267.207062,424.750935,580.000000,"Apartment (1 bedroom) in City Centre, Rent Per...",Zadar,EUR
521,63,26,500.000000,604.969309,750.000000,"Apartment (1 bedroom) in City Centre, Rent Per...",Zagreb,EUR
522,11,26,550.000000,631.250000,750.000000,"Apartment (1 bedroom) in City Centre, Rent Per...",Zaragoza (Saragossa),EUR


In [ ]:
#Output is DataFrame with 524 rows × 8 columns

In [72]:
#Drop column data_points, item_id, lowest_price and highest_price
df_new = df.drop(["data_points", 'item_id', 'lowest_price', 'highest_price'], axis=1, inplace=True)

In [76]:
#Drop column currency
df_clean = df.drop(['currency'], axis = 1)

In [92]:
#Add the column country
df_clean['country'] = df_clean['city'].map(city_count_dict)

In [93]:
df_clean

,average_price,item_name,city,country
0,561.444444,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany
1,726.963403,"Apartment (1 bedroom) in City Centre, Rent Per...",Aalborg,Denmark
2,640.494990,"Apartment (1 bedroom) in City Centre, Rent Per...",Aberdeen,United Kingdom
3,491.224523,"Apartment (1 bedroom) in City Centre, Rent Per...",Abidjan,Ivory Coast
4,674.859546,"Apartment (1 bedroom) in City Centre, Rent Per...",Accra,Ghana
...,...,...,...,...
519,992.284471,"Apartment (1 bedroom) in City Centre, Rent Per...",Yerevan,Armenia
520,424.750935,"Apartment (1 bedroom) in City Centre, Rent Per...",Zadar,Croatia
521,604.969309,"Apartment (1 bedroom) in City Centre, Rent Per...",Zagreb,Croatia
522,631.250000,"Apartment (1 bedroom) in City Centre, Rent Per...",Zaragoza (Saragossa),Spain


In [94]:
df_clean[df_clean['country'] == 'United States']

,average_price,item_name,city,country
14,1388.903623,"Apartment (1 bedroom) in City Centre, Rent Per...",Albuquerque,United States
27,1784.679543,"Apartment (1 bedroom) in City Centre, Rent Per...",Atlanta,United States
30,2145.734293,"Apartment (1 bedroom) in City Centre, Rent Per...",Austin,United States
60,1486.090513,"Apartment (1 bedroom) in City Centre, Rent Per...",Boise,United States
64,2580.265257,"Apartment (1 bedroom) in City Centre, Rent Per...",Boston,United States
77,2613.979011,"Apartment (1 bedroom) in City Centre, Rent Per...",Brooklyn,United States
84,1309.324441,"Apartment (1 bedroom) in City Centre, Rent Per...",Buffalo,United States
103,1821.536480,"Apartment (1 bedroom) in City Centre, Rent Per...",Charlotte,United States
107,1900.389651,"Apartment (1 bedroom) in City Centre, Rent Per...",Chicago,United States
111,1280.003780,"Apartment (1 bedroom) in City Centre, Rent Per...",Cincinnati,United States


In [100]:
df_clean[

519

In [101]:
df_clean['price_per_day'] = df_clean['average_price'] / 30

In [102]:
df_clean.head()

,average_price,item_name,city,country,price_per_day
0,561.444444,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany,18.714815
1,726.963403,"Apartment (1 bedroom) in City Centre, Rent Per...",Aalborg,Denmark,24.232113
2,640.494990,"Apartment (1 bedroom) in City Centre, Rent Per...",Aberdeen,United Kingdom,21.349833
3,491.224523,"Apartment (1 bedroom) in City Centre, Rent Per...",Abidjan,Ivory Coast,16.374151
4,674.859546,"Apartment (1 bedroom) in City Centre, Rent Per...",Accra,Ghana,22.495318


In [103]:
#price_per_day multiplicate with a factor 4 (avg factor for prices airbnb vs price_per_day )
df_clean['calc_price'] = df_clean['price_per_day'] * 4

In [104]:
df_clean.head()

,average_price,item_name,city,country,price_per_day,calc_price
0,561.444444,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany,18.714815,74.859259
1,726.963403,"Apartment (1 bedroom) in City Centre, Rent Per...",Aalborg,Denmark,24.232113,96.928454
2,640.494990,"Apartment (1 bedroom) in City Centre, Rent Per...",Aberdeen,United Kingdom,21.349833,85.399332
3,491.224523,"Apartment (1 bedroom) in City Centre, Rent Per...",Abidjan,Ivory Coast,16.374151,65.496603
4,674.859546,"Apartment (1 bedroom) in City Centre, Rent Per...",Accra,Ghana,22.495318,89.981273


In [106]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524 entries, 0 to 523
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   average_price  524 non-null    float64
 1   item_name      524 non-null    object 
 2   city           524 non-null    object 
 3   country        524 non-null    object 
 4   price_per_day  524 non-null    float64
 5   calc_price     524 non-null    float64
dtypes: float64(3), object(3)
memory usage: 24.7+ KB


## Push the DataFrame df_clean to sql

In [109]:
#Define schema for DBeaver
schema = 'capstone_travel_index'
engine = sf.get_engine()

In [110]:
#Import dataset as table to DBeaver, rename df_clean to country_avg_price
table_name = 'city_apart_prices'
if engine!=None:
    try:
        df_clean.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The city_apart_prices table was imported successfully.


In [ ]:
#The city_apart_prices table was imported successfully.